# `decorators.py`

This notebook tests the `decorators.py` module.

This module contains a set of decorators meant to change a function domain/codomain (probit decorators) or changing the coordinates in which the input/output of a function is expressed (celestial/cartesian decorators).

A note on all these decorators: they are meant to be used on class methods, therefore the first argument of the decorated function must be an instance of any class (in particular, for probit decorators, it must contain a `bounds` attribute) while the second is the point.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from corner import corner
from figaro.decorators import *

class foo:
    def __init__(self, bounds):
            self.bounds = np.array(bounds)
            
x_min  = 1
x_max  = 3
x      = np.linspace(x_min, x_max, 1002)[1:-1]
y      = np.linspace(-5, 5, 1000)
bounds = [[x_min, x_max]]
ref    = foo(bounds)

## Probit decorators

The probit decorators changes the domain or codomain of a function from natural space $\mathbb{N}$ to probit space $\mathbb{P}$ and vice versa.

### `antiprobit`

This decorator takes as input a function $f:\mathbb{P} \to \mathbb{R}^n$ and returns a function $g:\mathbb{N} \to \mathbb{R}^n$. It does so by first transforming the point from natural space to probit space with `transform_from_probit` and then evaluating the decorated function in the transformed point.

First check is to ensure that the decorated identity function works as coordinate changer:

In [ ]:
@antiprobit
def identity(ref, x):
    return x

plt.plot(y, identity(ref, y))
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.grid(True,dashes=(1,3))

A simple square law:

In [ ]:
@antiprobit
def square_decorated(ref, x):
    return 2*x**2

def square(ref, x):
    return 2*x**2


sq = square(ref, y)
sq_decorated = square_decorated(ref, y)

plt.plot(y, sq, lw = 0.7, label = '$\mathrm{Not\ decorated}$')
plt.plot(y, sq_decorated, lw = 0.7, label = '$\mathrm{Decorated}$')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc = 0, frameon = False)
plt.grid(True,dashes=(1,3))

### `probit`

This decorator takes as input a function $f:\mathbb{N} \to \mathbb{R}^n$ and returns a function $g:\mathbb{P} \to \mathbb{R}^n$. It does so by first transforming the point from natural space to probit space with `transform_to_probit` and then evaluating the decorated function in the transformed point.

As above, check that the decorated identity function works as coordinate changer:

In [ ]:
@probit
def identity(ref, x):
    return x

plt.plot(x, identity(ref, x))
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.grid(True,dashes=(1,3))

Square law:

In [ ]:
@probit
def square_decorated(ref, x):
    return 2*x**2

def square(ref, x):
    return 2*x**2


sq = square(ref, x)
sq_decorated = square_decorated(ref, x)

plt.plot(x, sq, lw = 0.7, label = '$\mathrm{Not\ decorated}$')
plt.plot(x, sq_decorated, lw = 0.7, label = '$\mathrm{Decorated}$')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc = 0, frameon = False)
plt.grid(True,dashes=(1,3))

### `from_probit`

This decorator takes as input a function $f:\mathbb{R}^n \to \mathbb{P}$ and returns a function $g:\mathbb{R}^n \to \mathbb{N}$. It does so by first evaluating the decorated function in the transformed point and then transforming the point from probit space to natural space with `transform_from_probit`.

As above, the first check ensures that for the identity function this works as a coordinate changer:

In [ ]:
@from_probit
def identity(ref, x):
    return x

plt.plot(y, identity(ref, y))
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.grid(True,dashes=(1,3))

Square law:

In [ ]:
@from_probit
def square_decorated(ref, x):
    return 2*x**2

sq_decorated = square_decorated(ref, y)

plt.plot(y, sq_decorated, lw = 0.7, label = '$\mathrm{Decorated}$')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.legend(loc = 0, frameon = False)
plt.grid(True,dashes=(1,3))

## Cartesian/celestial decorators

These decorators are used in the `threeDvolume.py` module in order to simplify the volume reconstruction by working in cartesian coordinates $x,y,z$ instead of celestial coordinates $\alpha,\delta,D_L$.

### `celestial`

We will show that the decorated identity function acts as a coordinate changer:

In [ ]:
@celestial
def identity(ref, x):
    return x

points = np.random.uniform(-100, 100, size = (200000,3))
celestial_points = identity(ref, points)
# Sphere inscribed in the cube
celestial_points = celestial_points[celestial_points[:,2] < 100]

c = corner(celestial_points, labels = ['$\\alpha$', '$\\delta$', '$D_L$'])

### `cartesian`

As above, the decorated identity function is expected to act as coordinate changer: 

In [ ]:
@cartesian
def identity(ref, x):
    return x

R = 100*np.sqrt(3)
n_samps = 1000000

phi = np.random.uniform(0,2*np.pi, size = n_samps)
costheta = np.random.uniform(-1,1, size = n_samps)
u = np.random.uniform(0,1, size = n_samps)
theta = np.arcsin(costheta)
r = R * np.cbrt(u)

points = np.array([phi, theta, r]).T
cartesian_points = identity(ref, points)
# Cube inscribed in the sphere
cartesian_points = cartesian_points[abs(cartesian_points[:,0]) < 100]
cartesian_points = cartesian_points[abs(cartesian_points[:,1]) < 100]
cartesian_points = cartesian_points[abs(cartesian_points[:,2]) < 100]

c = corner(cartesian_points, labels = ['$x$', '$y$', '$z$'])